In [21]:
!pip install sentence-transformers langchain langchain-community chromadb huggingface_hub sentence-transformers llama-cpp-python PyPDF2 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00


In [33]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from langchain.llms import LlamaCpp
from llama_cpp import Llama

In [34]:
current_dir = os.path.dirname(os.path.abspath("Files"))
file_path = os.path.join(current_dir, "git_tutorial.pdf")
persistent_directory = os.path.join(current_dir, "chroma_db")

In [35]:
if not os.path.exists(persistent_directory):
    print("Persistent directory does not exist, Initializing vector store...")

    if not os.path.exists(file_path):
      raise FileNotFoundError(f"File not found: {file_path}")

    loader = PyPDFLoader(file_path)
    documents = loader.load()

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)

    print("\n*** DOcument Chunks Information ***")
    print(f"Number of document chunks: {len(docs)}")
    print(f"Sample chunk:\n{docs[0].page_content}")

    print("\n*** Loading Local Embedding Model... ***")
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
        )

    print("\n*** Creating Vector Store... ***")
    vectorstore = Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        persist_directory=persistent_directory
        )
    print("\n*** Finished creating vectorstore ***")

else:
    print("Vector store already exists . No need to initialize")


Vector store already exists . No need to initialize


In [36]:
persistent_directory = "./chroma_db"

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = Chroma(persist_directory=persistent_directory, embedding_function=embedding_model)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [37]:
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(
    repo_id="TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF",
    filename="tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf",
)

In [38]:
print(f"Model downloaded to: {model_path}")

Model downloaded to: /root/.cache/huggingface/hub/models--TheBloke--TinyLlama-1.1B-Chat-v1.0-GGUF/snapshots/52e7645ba7c309695bec7ac98f4f005b139cf465/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf


In [39]:
from llama_cpp import Llama

llm = Llama(
    model_path = model_path,
    n_ctx=2048,
    n_threads=4,
    n_batch=8
)

llama_model_loader: loaded meta data with 23 key-value pairs and 201 tensors from /root/.cache/huggingface/hub/models--TheBloke--TinyLlama-1.1B-Chat-v1.0-GGUF/snapshots/52e7645ba7c309695bec7ac98f4f005b139cf465/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tinyllama_tinyllama-1.1b-chat-v1.0
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loade

In [49]:
llm_wrapper = LlamaCpp(
    model_path=model_path,
    temperature=0.5,
    max_tokens=512,
    n_ctx=2048,
    verbose=True
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm_wrapper,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

llama_model_loader: loaded meta data with 23 key-value pairs and 201 tensors from /root/.cache/huggingface/hub/models--TheBloke--TinyLlama-1.1B-Chat-v1.0-GGUF/snapshots/52e7645ba7c309695bec7ac98f4f005b139cf465/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tinyllama_tinyllama-1.1b-chat-v1.0
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loade

In [50]:
query = "How do you create a Git branch?"
response = qa_chain.invoke({"query": query})
print(response["result"])

llama_perf_context_print:        load time =  123127.07 ms
llama_perf_context_print: prompt eval time =  123126.88 ms /  1530 tokens (   80.48 ms per token,    12.43 tokens per second)
llama_perf_context_print:        eval time =   84873.39 ms /   511 runs   (  166.09 ms per token,     6.02 tokens per second)
llama_perf_context_print:       total time =  208922.99 ms /  2041 tokens



1. Git Branches: A branch is a copy of your current repository, which you can work on 
without affecting the main branch. It's like a separate project. You create a new branch by creating a new directory in your repo and cloning that branch to it. 2. Creating a new branch:
- Open Git Bash or Terminal (Windows) or Command Prompt/PowerShell (MacOS) 
- Enter the command: git checkout -b <branch_name> 3. Check out a branch:
- Open Git Bash or Terminal (Windows) or Command Prompt/PowerShell (MacOS)
- Enter the command: git checkout <branch_name> 4. Merge a branch into master:
- Open Git Bash or Terminal (Windows) or Command Prompt/PowerShell (MacOS)
- Enter the command: git merge -m "Merged branch" <branch_to_merge>
- Replace 'master' with the name of your master branch. 5. Removing a branch:
- Open Git Bash or Terminal (Windows) or Command Prompt/PowerShell (MacOS)
- Enter the command: git branch -d <branch_name> 6. Checking out an older commit:
- Open Git Bash or Terminal (Windows) or Co